In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
import sys
import torch
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

import neural_network_lyapunov
import neural_network_lyapunov.relu_system as relu_system
import neural_network_lyapunov.lyapunov as lyapunov
import neural_network_lyapunov.encoders as encoders
import neural_network_lyapunov.pybullet_data_generation as pybullet_data_generation
import neural_network_lyapunov.dynamics_learning as dynamics_learning

## Load settings and models

In [ ]:
folder = "pendulum_1013"

sys.path.insert(0, os.path.join(os.path.dirname(neural_network_lyapunov.__file__), "..", "notebooks", folder))
import options
opt = dynamics_learning.DynamicsLearningOptions(options.default)

In [ ]:
x_data = torch.load(os.path.join(folder, "x_data"))
x_next_data = torch.load(os.path.join(folder, "x_next_data"))
X_data = torch.load(os.path.join(folder, "X_data"))
X_next_data = torch.load(os.path.join(folder, "X_next_data"))
X_rollouts = torch.load(os.path.join(folder, "X_rollouts"))
x_rollouts = torch.load(os.path.join(folder, "x_rollouts"))

In [ ]:
x_train_dataloader = torch.load(os.path.join(folder, "x_train_dataloader"))
x_validation_dataloader = torch.load(os.path.join(folder, "x_validation_dataloader"))
X_train_dataloader = torch.load(os.path.join(folder, "X_train_dataloader"))
X_validation_dataloader = torch.load(os.path.join(folder, "X_validation_dataloader"))

In [ ]:
learners = dict()
for variant in options.variants.keys():
    opt.set_options(options.variants[variant])
    
    dyn_nn_model = dynamics_learning.get_ff_network(opt.dtype, opt.z_dim, opt.z_dim, opt.dyn_nn_width, opt.dyn_nn_depth)
    lyap_nn_model = dynamics_learning.get_ff_network(opt.dtype, opt.z_dim, 1, opt.lyap_nn_width, opt.lyap_nn_depth)

    relu_sys = relu_system.AutonomousReLUSystemGivenEquilibrium(opt.dtype, opt.z_lo, opt.z_up,
                                                                dyn_nn_model, opt.z_equilibrium)
    lyap = lyapunov.LyapunovDiscreteTimeHybridSystem(relu_sys, lyap_nn_model)
#     encoder = opt.encoder_class(opt.z_dim, opt.image_width, opt.image_height, opt.grayscale)
#     decoder = opt.decoder_class(opt.z_dim, opt.image_width, opt.image_height, opt.grayscale)

#     dyn_learner = dynamics_learning.LatentSpaceDynamicsLearning(X_train_dataloader, X_validation_dataloader,
#                                                                 relu_sys, lyap, opt,
#                                                                 encoder, decoder,
#                                                                 use_bce=opt.use_bce,
#                                                                 use_variational=opt.use_variational)
    dyn_learner = dynamics_learning.StateSpaceDynamicsLearning(x_train_dataloader, x_validation_dataloader,
                                                               relu_sys, lyap, opt)

    dyn_learner.relu_system.dynamics_relu = torch.load(os.path.join(folder, variant, "dynamics"))
    dyn_learner.lyapunov.lyapunov_relu = torch.load(os.path.join(folder, variant, "lyapunov"))
#     dyn_learner.encoder = torch.load(os.path.join(folder, variant, "encoder"))
#     dyn_learner.decoder = torch.load(os.path.join(folder, variant, "decoder"))
    
    learners[variant] = dyn_learner

## Comparing models

In [ ]:
# computing adversarial samples
z_adv_pos, z_adv_der = learners["unstable"].adversarial_samples(optimal=opt.lyapunov_loss_optimal)
print(z_adv_pos)
print(z_adv_der)

In [ ]:
pbsg = pybullet_data_generation.PybulletSampleGenerator(opt.world_cb, opt.joint_space, image_width=opt.image_width, image_height=opt.image_height,
                                                        grayscale=opt.grayscale, dtype=opt.dtype,
                                                        camera_eye_position=opt.camera_eye_position,
                                                        camera_target_position=opt.camera_target_position,
                                                        camera_up_vector=opt.camera_up_vector)

In [ ]:
# z_adv = z_adv_pos
z_adv = z_adv_der
i = np.random.choice(z_adv.shape[0], 1)[0]
x0 = z_adv[i, :]

# i = np.random.choice(x_data.shape[0], 1)[0]
# x0 = x_data[i, :]

print(x0)
for name in learners.keys():
    dyn_learner = learners[name]
    x_traj, V_traj = dyn_learner.rollout(x0, opt.validation_rollouts_N)
    plt.plot(x_traj)
    _, x_traj_exp = pbsg.generate_rollout(x0, opt.dataset_dt, opt.validation_rollouts_N)
    plt.plot(x_traj_exp)
    plt.legend(['predicted', 'predicted', 'expected', 'expected'])
    plt.title(name)
    plt.show()
    plt.plot(V_traj)
    plt.title(name)
    plt.show()

In [ ]:
legend = []
for name in learners.keys():
    dyn_learner = learners[name]
#     rollout_loss = dyn_learner.rollout_validation(X_rollouts, device='cuda')
    rollout_loss = dyn_learner.rollout_validation(x_rollouts[:75], device='cpu')
    plt.plot(rollout_loss)
    legend.append(name)
#     plt.title(name)
#     plt.show()
plt.legend(legend)
plt.show()